[Notebook of task](https://github.com/DistributedSystemsGroup/Algorithmic-Machine-Learning/blob/master/Challenges/Anomaly_Detection/anomaly_detection_challenge.ipynb)

In [2]:
# Package for scalable bayesian rule lists
!pip3 install --user 'fim'
!pip3 install --user 'pysbrl'

    100% |################################| 348kB 1.2MB/s 
  Running setup.py bdist_wheel for fim ... done
  Stored in directory: /mnt/workspace/.cache/pip/wheels/5c/1c/94/b96c6b9a2eb858e26a675f86a908abfa53a593185b1c058823
Successfully built fim
You are using pip version 18.0, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Elementary
import os
import sys
import re
import random
import matplotlib
import implicit
import warnings
from tqdm import tqdm

# For elementary data manipulation
import pandas as pd
import numpy as np

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For scalable bayesian rule lists
import pysbrl

# Import dataframe and cast names, datatypes and NaNs
from names import column_names, labels
basepath = "/mnt/datasets/anomaly/"
dataDF = pd.read_csv(basepath + 'data.csv', delimiter=";", header=None, names=column_names)
pure_dataDF = dataDF.drop(labels, axis=1)
anomaliesDF = dataDF.filter(labels, axis=1) 

anomaliesDF_with_zerNA = anomaliesDF.fillna(0) # Fill NaNs with 0s, considering them as "not an anomaly"
anomaliesDF_with_negNA = anomaliesDF.fillna(-1) # Fill NaNs with -1 considering them as a separate class for the classifier.
pure_dataDF_with_negNA = pure_dataDF.fillna(-1)

Nyttige artikler om stratified shuffle split
* [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)
* [Visualizing cross-validation behavior in scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py)
* [User guide: cross validation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

# Pre-processing

## Remove data corresponding to one NaN column

In [10]:
def create_binary_classification(puredataDF, anomaliesDF, label):
    y = anomaliesDF[label]
    indices_nan_labels = list(y.index[y.isnull()])
    
    X_mod = puredataDF.drop(indices_nan_labels, axis=0)
    y_mod = y.drop(indices_nan_labels)
    
    X_mod = X_mod.reset_index().drop('index', axis=1)
    y_mod = y_mod.reset_index().drop('index', axis=1)
    return X_mod, y_mod

# TEST ------------------------------------------------------
X_temp = pure_dataDF_with_negNA.drop('Date',axis=1)
X, y = create_binary_classification(X_temp, anomaliesDF, 'Check1')

print('X_temp.shape:', X_temp.shape, '\t', 'X.shape:', X.shape)

X_temp.shape: (287031, 35) 	 X.shape: (262520, 35)


## Stratified-shuffle-split function

This function will split the a given dataframe X, and corresponding label-series y (only one column), into train, validation and test sets such that the distribution of the different labels is retained in the different data sets.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# This function splits the dataset into 0.6 train, 0.2 val and 0.2 test sets ONLY SINGLE LABEL
def train_val_test_split(X, y, seed):
    # This generator splits the OG dataset into train and test sets
    sss_train_test = StratifiedShuffleSplit(n_splits = 1, 
                                   test_size = 0.2, 
                                   train_size = 0.8, 
                                   random_state = seed)

    # This generator splits the newly created train-set into train and validate sets
    sss_train_val = StratifiedShuffleSplit(n_splits = 1, 
                                   test_size = 0.25, 
                                   train_size = 0.75, 
                                   random_state = seed)

    for train_index, test_index in sss_train_test.split(X,y):
        X_temp = X.loc[train_index, :]
        y_temp = y.loc[train_index, :]
        X_test = X.loc[test_index, :]
        y_test = y.loc[test_index, :]

    for train_index, test_index in sss_train_val.split(X_temp,y_temp):
        X_train = X_temp.loc[train_index, :]
        y_train = y_temp.loc[train_index, :]
        X_val = X_temp.loc[test_index, :]
        y_val = y_temp.loc[test_index, :]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

# TEST ---------------------------------------------------------------------------------------
seed = 42
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y, seed)

print('####### Y TRAIN #######\n', y_train.describe())
print('\n######## Y VAL ########\n', y_val.describe())
print('\n####### Y TEST ########\n', y_test.describe())

## Scalable Bayesian Rule Lists
### [github repo](https://github.com/myaooo/pysbrl)

In [1]:
# Save dataframes to the format the SBRL library requires
categorical2pysbrl_data(
    X_train_mod,
    y_train_mod,
    'X.out',
    'y.label',
    method='eclat',
    supp=0.05,
    zmin=1,
    zmax=3)

NameError: name 'categorical2pysbrl_data' is not defined

In [ ]:
# Use SBRL Library from https://github.com/myaooo/pysbrl
rule_ids, outputs, rule_strings = pysbrl.train_sbrl("data/ttt_train.out", 
                                                    "data/ttt_train.label", 
                                                    20.0, 
                                                    eta=2.0, 
                                                    max_iters=2000, 
                                                    nchain=10, 
                                                    alphas=[1,1])